In [1]:
import pandas as pd

### Tags

In [2]:
tags = pd.read_csv("Tags.csv", engine='python')

tagsAndParents = tags[["Name", "Id"]]
tagsAndParents = tagsAndParents.rename(columns={"Id": "ParentId", "Name": "ParentTagName"})

tagsAndParents = pd.merge(tags, tagsAndParents, left_on="ParentTagId", right_on="ParentId")
tagsAndParents = tagsAndParents.drop(['ParentTagId', 'ParentId'], axis=1)

tagsAndParents.head()

,Id,Name,Slug,FullPath,Description,DatasetCount,CompetitionCount,KernelCount,ParentTagName
0,1115,websites,web-sites,subject > science and technology > internet > ...,"Websites, like this one, are collections of co...",250,0,209,internet
1,1220,search engines,search-engines,subject > science and technology > internet > ...,NaN,72,0,72,internet
2,12117,mobile and wireless,mobile-web,subject > science and technology > internet > ...,NaN,601,4,149,internet
3,16486,email and messaging,email-and-messaging,subject > science and technology > internet > ...,NaN,2129,1,129,internet
4,16489,online communities,online-communities,subject > science and technology > internet > ...,NaN,6829,3,1232,internet


### Dataset tags

In [3]:
datasetTags = pd.read_csv("DatasetTags.csv", engine='python')
datasetTags.head()

,Id,DatasetId,TagId
0,1879,2234,1115
1,2182,2482,1115
2,2408,2700,1115
3,3756,8095,1115
4,5942,22268,1115


## Generate word clouds over time

In [4]:
years = range(2018, 2023)

In [5]:
def loadDataset(year):
  datasets = pd.read_csv('TopDatasets' + str(year) + '.csv', engine='python')

  return datasets

In [6]:
def mergeDatasetAndTags(datasets, datasetTags): 
  datasetsAndTags = pd.merge(datasetTags[["DatasetId", "TagId"]], tagsAndParents, left_on="TagId", right_on="Id")
  datasetsAndTags = datasetsAndTags.drop(['Id', 'TagId'], axis=1)

  datasetsAndTags = pd.merge(datasets, datasetsAndTags, left_on="Id", right_on="DatasetId")
  datasetsAndTags = datasetsAndTags.drop(['DatasetId'], axis=1)

  return datasetsAndTags

In [7]:
def countTags(datasetsAndTags):
  datasetTagAndValue = datasetsAndTags.groupby(['Name', 'ParentTagName'])['Name'].count()
  return datasetTagAndValue

In [8]:
for year in years:
  datasets = loadDataset(year)
  datasetsAndTags = mergeDatasetAndTags(datasets, datasetTags)
  datasetTagAndValue = countTags(datasetsAndTags)

  datasetTagAndValue.to_csv('DatasetTagAndValue' + str(year) + '.csv')